<a href="https://colab.research.google.com/github/nestoredduardo/two-year-churn-predictive-model/blob/main/Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Scientist Hackathon**

# 1. Problem

# 2. Obtain Data

In [1]:
import pandas as pd

In [2]:
tb_clients_path = '/content/drive/MyDrive/Jobs/KIN/clients_table.txt'
tb_credit_score_path = '/content/drive/MyDrive/Jobs/KIN/credit_score_table.txt'
tb_products_path = '/content/drive/MyDrive/Jobs/KIN/products_table.txt'
tb_transactions_path = '/content/drive/MyDrive/Jobs/KIN/transactions_table.txt'

In [3]:
df_clients = pd.read_csv(tb_clients_path)
df_credit_score = pd.read_csv(tb_credit_score_path)
df_products = pd.read_csv(tb_products_path)
df_transactions = pd.read_csv(tb_transactions_path)

# 3. Scrub Data

In [4]:
df_clients.head()

,CustomerId,Surname,Geography,Gender,HasCrCard,IsActiveMember,EstimatedSalary,application_date,exit_date,birth_date
0,15745584,EIRLS,Germany,Female,0.0,1.0,0.00,2018-12-14,NaN,1997-09-18
1,14990118,MOLOCK,Italy,Male,1.0,0.0,121219.28,2019-01-08,NaN,1980-08-03
2,14733224,PAWLUCH,Italy,Female,1.0,1.0,159663.59,2012-08-01,2013-08-09,1977-08-19
3,14648573,NALLS,Spain,Male,1.0,0.0,140827.98,2019-06-19,NaN,1979-02-27
4,15365443,EBERLE,Italy,Male,1.0,0.0,35521.28,2014-01-26,2015-12-04,1972-12-21


In [5]:
df_clients.shape

(1545000, 10)

## Desired Population:
a. Contracts from 2015 onwards.

b. Operations in Italy were closed this year (2019).


c. Take out clients with more than 75% of their info missing.

d. Each client should only have one contract in the database, no more.

e. Remember that for a client to be eligible, it must have at least two years of information within the company (even if it canceled the product).

### a. Contracts from 2015 onwards.

In [6]:
df_clients_desired = df_clients[df_clients['application_date'] >= '2015-01-01']
print(df_clients_desired.head())
print(df_clients_desired.shape)

   CustomerId   Surname Geography  ... application_date   exit_date  birth_date
0    15745584     EIRLS   Germany  ...       2018-12-14         NaN  1997-09-18
1    14990118    MOLOCK     Italy  ...       2019-01-08         NaN  1980-08-03
3    14648573     NALLS     Spain  ...       2019-06-19         NaN  1979-02-27
5    15638124  BRASHERS     Italy  ...       2018-02-23         NaN  1983-01-13
7    14523468  LASKOSKI     Spain  ...       2017-12-28  2018-11-19  1972-10-30

[5 rows x 10 columns]
(623242, 10)


### b. Operations in Italy were closed this year (2019).

In [7]:
df_clients_desired = df_clients_desired[df_clients_desired['Geography'] != 'Italy']
print(df_clients_desired.head())
print(df_clients_desired.shape)

    CustomerId   Surname Geography  ... application_date   exit_date  birth_date
0     15745584     EIRLS   Germany  ...       2018-12-14         NaN  1997-09-18
3     14648573     NALLS     Spain  ...       2019-06-19         NaN  1979-02-27
7     14523468  LASKOSKI     Spain  ...       2017-12-28  2018-11-19  1972-10-30
14    15165393  LABIANCA     Spain  ...       2018-02-22  2019-06-11  1974-07-11
15    14611239     DOKKA    France  ...       2019-02-24         NaN  1986-04-26

[5 rows x 10 columns]
(487424, 10)


### c. Take out clients with more than 75% of their info missing.

In [8]:
df_clients_desired.isna().sum()

CustomerId               0
Surname              23362
Geography            23349
Gender               23349
HasCrCard            23349
IsActiveMember       23349
EstimatedSalary      23349
application_date         0
exit_date           382626
birth_date           23349
dtype: int64

In [9]:
number_info_missing = 75/100 * 9
print(number_info_missing)

6.75


In [10]:
df_clients_desired = df_clients_desired[df_clients_desired.isnull().sum(axis=1) < number_info_missing]
print(df_clients_desired.head())
print(df_clients_desired.shape)

    CustomerId   Surname Geography  ... application_date   exit_date  birth_date
0     15745584     EIRLS   Germany  ...       2018-12-14         NaN  1997-09-18
3     14648573     NALLS     Spain  ...       2019-06-19         NaN  1979-02-27
7     14523468  LASKOSKI     Spain  ...       2017-12-28  2018-11-19  1972-10-30
14    15165393  LABIANCA     Spain  ...       2018-02-22  2019-06-11  1974-07-11
15    14611239     DOKKA    France  ...       2019-02-24         NaN  1986-04-26

[5 rows x 10 columns]
(464075, 10)


### d. Each client should only have one contract in the database, no more.

In [11]:
df_clients_desired.drop_duplicates(subset=['CustomerId'],keep= 'first', inplace= True)
print(df_clients_desired.head())
print(df_clients_desired.shape)

    CustomerId   Surname Geography  ... application_date   exit_date  birth_date
0     15745584     EIRLS   Germany  ...       2018-12-14         NaN  1997-09-18
3     14648573     NALLS     Spain  ...       2019-06-19         NaN  1979-02-27
7     14523468  LASKOSKI     Spain  ...       2017-12-28  2018-11-19  1972-10-30
14    15165393  LABIANCA     Spain  ...       2018-02-22  2019-06-11  1974-07-11
15    14611239     DOKKA    France  ...       2019-02-24         NaN  1986-04-26

[5 rows x 10 columns]
(450562, 10)


### e. Remember that for a client to be eligible, it must have at least two years of information within the company (even if it canceled the product).

In [12]:
today_date = '30/11/2019'

In [13]:
application_date = pd.to_datetime(df_clients_desired['application_date'])
df_clients_desired['exit_date'] = df_clients_desired['exit_date'].fillna(today_date)
exit_date = pd.to_datetime(df_clients_desired['exit_date'])

In [14]:
df_clients_desired['application_date'] = application_date
df_clients_desired['exit_date'] = exit_date
df_clients_desired.head()

,CustomerId,Surname,Geography,Gender,HasCrCard,IsActiveMember,EstimatedSalary,application_date,exit_date,birth_date
0,15745584,EIRLS,Germany,Female,0.0,1.0,0.00,2018-12-14,2019-11-30,1997-09-18
3,14648573,NALLS,Spain,Male,1.0,0.0,140827.98,2019-06-19,2019-11-30,1979-02-27
7,14523468,LASKOSKI,Spain,Female,1.0,0.0,158161.23,2017-12-28,2018-11-19,1972-10-30
14,15165393,LABIANCA,Spain,Male,1.0,1.0,2612.65,2018-02-22,2019-06-11,1974-07-11
15,14611239,DOKKA,France,Male,0.0,1.0,72210.60,2019-02-24,2019-11-30,1986-04-26


In [15]:
df_clients_desired = df_clients_desired[df_clients_desired['exit_date'] - df_clients_desired['application_date'] >= '730 days']

In [16]:
print(df_clients_desired.head())
print(df_clients_desired.shape)

      CustomerId        Surname  ...  exit_date  birth_date
75      15610711       Eluemuno  ... 2019-11-27  1976-10-21
484     15594720          Scott  ... 2019-11-30  1982-05-14
901     15706552  Odinakachukwu  ... 2019-10-02  1979-09-09
1440    15594084       Anderson  ... 2019-09-07  1993-06-08
1543    15756871          Capon  ... 2018-04-29  1976-04-20

[5 rows x 10 columns]
(7963, 10)


# 4. Explore Data

### Age

In [17]:
dataset = df_clients_desired['CustomerId']
df_dataset = pd.DataFrame(data = dataset)

In [18]:
from datetime import datetime, date

def age(born):
    born = datetime.strptime(born, "%Y-%m-%d").date()
    today = date.today()
    return today.year - born.year - ((today.month, 
                                      today.day) < (born.month, 
                                                    born.day))

In [19]:
df_dataset['Age'] = df_clients_desired['birth_date'].apply(age)
df_dataset.set_index('CustomerId')

,CustomerId,Age
75,15610711,44
484,15594720,39
901,15706552,42
1440,15594084,28
1543,15756871,45


### Number of products

In [20]:
products_count = df_products['CustomerId'].value_counts()
df_products_count = pd.DataFrame(data = products_count, columns = 'NumProducts')

In [21]:
df_products_count.head()

,CustomerId
14678892,4
15932667,4
15550521,4
14976520,4
15742810,4


### Balance at the moment of application

In [22]:
balance = df_transactions.groupby('CustomerId')['Value'].sum()
df_balance = pd.DataFrame(data = balance)
df_balance.head()

,Value
CustomerId,
14500000,0.00
14500001,51405.61
14500002,120763.50
14500003,149896.71
14500004,143061.53


### Credit bureau Scores

In [23]:
def get_month(appDate):
    return appDate.month, appDate.year

In [24]:
df_client_app_date = pd.DataFrame(df_clients_desired['CustomerId'])
df_client_app_date['Month'], df_client_app_date['Year'] = zip(*df_clients_desired['application_date'].apply(get_month))

In [25]:
df_credit_score_date = pd.to_datetime(df_credit_score['Date'])
df_credit_score['Date'] = df_credit_score_date
df_credit_score['Month'], df_credit_score['Year']= zip(*df_credit_score['Date'].apply(get_month))

In [48]:
def getScore(row):
    score_by_id = df_credit_score[df_credit_score['CustomerId'] == row.CustomerId]
    score_by_month = score_by_id[(score_by_id['Year'] == row['Year']) & (score_by_id['Month'] == row['Month'])]
    return score_by_month['Month']

In [54]:
df_client_score= df_client_app_date.apply(getScore, axis=1)
df_client_score.head()

AttributeError: ignored

In [53]:
df_client_app_date

,CustomerId,Month,Year
75,15610711,3,2017
484,15594720,11,2017
901,15706552,2,2016
1440,15594084,3,2016
1543,15756871,5,2015
...,...,...,...
1543267,15613180,2,2016
1543548,15689152,4,2016
1543594,15787204,8,2017
1544413,15778320,10,2016


In [ ]:
df_products.head()
df_transactions.head()
df_credit_score.head()

In [ ]:
df_products['CustomerId'].value_counts()

In [ ]:
df_transactions[df_transactions['CustomerId'] == 15346641]

In [ ]:
df_credit_score[df_credit_score['CustomerId'] == 15610711]

In [ ]:
df_clients_desired[df_clients_desired['CustomerId'] == 15610711]

# 5. Model Data

# 6. Interpreting Data